In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
#takes as input argument the profile for the computation of fft

def fft_for_window(profile: xr.core.dataarray.DataArray, 
                   step=16e6, #distance on the frequency axis
                   min_band=4e9, 
                   max_band=6e9,
                   perc_off=0.1): #percentage of data to discard, crescendo should have 0.2
    
    lim_inf = profile[profile["Frequency"].data >= min_band]
    lim = lim_inf[lim_inf["Frequency"].data <= max_band]

    #compute fft
    fft = np.fft.fft(lim)
    magnitude = np.abs(fft)

    #x axis range
    frequencies = np.linspace(min_band, max_band, len(lim))
    time = frequencies
    dt = np.mean(np.diff(time))
    fft_frequencies = np.fft.fftfreq(len(lim), d=dt)


    #calculate max(fft) and frequency
    massimo_trasf = np.max(magnitude[int(perc_off*len(fft_frequencies)//2):len(fft_frequencies)//2])
    i0 = np.argmax(magnitude[int(perc_off*len(fft_frequencies)//2):len(fft_frequencies)//2])
    freq_trasf = 1/fft_frequencies[int(perc_off*len(fft_frequencies)//2):len(fft_frequencies)//2][i0]
    print(f"Max(fft) = {massimo_trasf}\nTime = {1/freq_trasf}\nFreq = {freq_trasf}")

    #find window
    window_opt = (int(2*freq_trasf/step) if int(2*freq_trasf/step)%2 == 1 else int(2*freq_trasf/step)-1)


    fig, ax = plt.subplots(2, 1, figsize=[10, 5], dpi=200)

    #profile plot
    ax[0].plot(time, lim, color="red", label="Profile")
    ax[0].set_xlabel("Frequency (Hz)") 
    ax[0].set_ylabel("|S21| / dB")
    ax[0].legend()
    ax[0].grid()

    #fft plot
    ax[1].plot(fft_frequencies[int(perc_off*len(fft_frequencies)//2):len(fft_frequencies)//2], magnitude[int(perc_off*len(fft_frequencies)//2):len(fft_frequencies)//2])
    ax[1].set_xlabel("Time (s)")
    ax[1].set_ylabel("Magnitude")
    ax[1].legend()
    ax[1].grid()

    fig.tight_layout()
    plt.show()

    return window_opt, massimo_trasf, freq_trasf
